# Verslag modelleren

*gemaakt door Kees Brouwer en Robbert van Nieuwkerk*

Eerst het importeren van de benodigde libraries

In [1]:
# importeren libs
import numpy as np
import pandas as pd
# import datetime
from itertools import product
# import itertools
import pickle
import copy

# import sys
# import dovpanda
import Importeer 
import Hulpfuncties
import CrewCombined
import Dataframe
import Controleren
from Class import MogelijkeCombinaties, Rooster

In [2]:
# print(sys.getsizeof(dfSprint))

er moet ook nog een keer worden uitgelegd wat er in welk df staat en wat er met ieder df is gedaan voordat het geimporteerd is. !TODO

Het gegeven excelbestand wat de informatie vat, is op een aantal punten aangepast voordat het is geimporteerd. 
Het bestand heeft voor iedere skill een aparte kolom. Mocht de skill nodig zijn voor de gegeven opdracht, dan staat er een 'x' in deze kolom. Ditzelfde is gedaan met de skills van medewerkers, de dagen waarop de medewerkers werken en voor de kamers waarin de opdrachten moeten worden uitgevoerd. Voor de additional tasks zijn er een aantal opdracht die in iedere sprint moeten worden uitgevoerd. Deze zijn opgesplitst, voor iedere sprint een aparte lijn. Hierbij is ook een kolom toegevoegd met het aantal frequenties voor deze opdracht. Ook is er een kolom toegevoegd bij het tabblad 'additional tasks' waarin staat aangegeven met hoeveel personen het minimaal moet worden uitgevoerd.
Hieronder worden de verschillende tabbladen ingelezen en toegekent aan een dataframe.
Verdere bewerkingen op de data worden gedaan in dit document.


In [3]:
# CONSTANTE
SPRINT = 'S1'


# ik weet niet hoe vaak deze nog wordt gebruikt.
rangeSkills = range(1,14)
rangeWorkdays = range(14,19)
rangeRooms = range(14,20)

In [4]:
# Importeren van verschillende sheets. Dit wordt gedaan met functies.
# in deze functies wordt het ook in het goede format gezet.

dfCrew = Importeer.Crew()
dfRooms = Importeer.Rooms()
dfSprint = Importeer.dfSprint(SPRINT)

In [5]:
# CrewCombined.MakeCrewCombined(dfCrew) # voor het maken van dfCrewcombined.csv (duurt even)
dfCrewCombined = pd.read_csv('CrewCombined.csv') # lezen van csv zodra die in wdir staat.

**Belangrijke info voor het plannen**
- De ingehuurde mensen hebben skill 1 en 8 
- kort inhuren is maximaal 60 uur
- het bedrijf is van 9-18 open
- een crewlid kan niet meer dan 3 uur achter elkaar werken, daarna een half uur pauze
- een crewlid werkt nooit meer dan 7 uur op een dag
- kan een 'acquire skill' kan voor ieder crewlid worden gedaan, geen beperking in aantal 
- crewleden moet te samen aan de skills komen voor de opdracht.
- Mensen die skill één niet hebben, hoeven deze niet te leren
- Een crewleden kunnen alleen aan eengesloten uren aan een project task werken. Dit mag niet worden doorbroken met een andere task. Met additional tasks mag dit wel. 
- iedereen moet tegelijk aan het project werken als ze op hetzelfde project staan.

Moet A volledig worden afgerond of maar 1 keer van het aantal frequenties? Moet volledig worden ingepland

We maken een soort kalender, startend op 6 januari 2020 en geen weekend dagen erin.
Hiervan maken we een dataframe met de volgende kolommen:
- datum
- een kolom met dataframes met de volgende punten:
    - tijd per halfuur
    - persoon
    - aantal uren te gaan
    - hierin staan de projecten met de kamer (is dit zo?)
- Of het een werkdag is (T/F)
- In welke sprint de dag zich bevindt.

In [6]:
dfKalenderCrew = Dataframe.LegeKalender()

In [7]:
dfKalenderRoom = pickle.loads(pickle.dumps(dfKalenderCrew)) # het maken van een kalender van de kamers, tot dit punt waren de df's hetzelfde

In [8]:
# dfSprint.iloc[:,26]= dfSprint.apply(lambda rij : Hulpfuncties.ToevoegenMogelijkeCombinaties(rij, 
#                                                     dfSprint, dfCrewCombined), axis = 1)

# dfSprint.iloc[:,27] = dfSprint['Frequency']*dfSprint['MinReqCrew']

In [9]:
# # het schrijven van dfSprint naar een pickle bestand
# filehandler = open('dfSprint1_begin.obj', 'wb') 
# pickle.dump(dfSprint, filehandler)

In [10]:
# het lezen van dfSprint, met meerdere keren runnen een heel stuk sneller
filehandler = open('dfSprint1_begin.obj', 'rb') 
dfSprint = pickle.load(filehandler)

In [11]:
dfSprint = dfSprint.sort_values(['AantalMogelijkheden'], ascending= [False])
dfSprint = dfSprint.reset_index(drop=True)

In [12]:
Eisen = dfSprint.Eis.unique().tolist()
Eisen.remove('no')

dfSprint = (dfSprint.loc[dfSprint['Task'].isin(Eisen)].append(dfSprint.loc[~dfSprint['Task'].isin(Eisen)])).reset_index(drop=True)

In [13]:
StandaardRooster = Dataframe.RoosterCrew(dfCrew)
dfRoosterKamer = Dataframe.RoosterKamer(dfRooms)

In [14]:
# inplannen stand-up op dinsdagochtend
dfRoosterDinsdag = copy.deepcopy(StandaardRooster)
dfRoosterDinsdag.iloc[1:7,1:14] = "weekly stand-up"
dfRoosterDinsdag.iloc[0,1:14] = 4

dfRoosterKamerDinsdag = copy.deepcopy(dfRoosterKamer)
dfRoosterKamerDinsdag.loc[0:5, 'workshop A'] ="weekly stand-up"

In [15]:
for i in range(0,5): # het maken van de lege roosters voor week 1
    if i== 1:
        rooster = copy.deepcopy(dfRoosterDinsdag)
        rooster.iloc[0,1:] = (np.array(rooster.iloc[0,1:])* np.array(dfCrew.iloc[:,14+i])).tolist() # het maken van de lijst urentegaan met de vermenigvuldiging van de lijst met dagen dat iedereen werkt en een lijst met alleen maar 7's, resultaat: 7 als iemand werkt, als iemand niet werkt, 0.
        dfKalenderCrew.DagRooster[i] = Rooster(rooster)
        dfKalenderRoom.DagRooster[i] = Rooster(copy.deepcopy(dfRoosterKamerDinsdag))
    else:
        rooster = copy.deepcopy(StandaardRooster)
        rooster.iloc[0,1:] = (np.array(rooster.iloc[0,1:])* np.array(dfCrew.iloc[:,14+i])).tolist() # het maken van de lijst urentegaan met de vermenigvuldiging van de lijst met dagen dat iedereen werkt en een lijst met alleen maar 7's, resultaat: 7 als iemand werkt, als iemand niet werkt, 0.
        dfKalenderCrew.DagRooster[i] = Rooster(rooster)
        dfKalenderRoom.DagRooster[i] = Rooster(copy.deepcopy(dfRoosterKamer))
    
for i in range(1,11): # het kopieren van de roosters van week 1 voor de laatste weken in de sprint (9-10 weken)
    for j in range(0,5):
        dfKalenderCrew.DagRooster[i*5 + j] = copy.deepcopy(dfKalenderCrew.DagRooster[j])
        dfKalenderRoom.DagRooster[i*5 + j] = copy.deepcopy(dfKalenderRoom.DagRooster[j])

C:\Users\Robbert\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Robbert\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


!TODO opdrachten over twee dagen

In [16]:
dfSprintAddprep, dfSprintAddpost = Importeer.AdditionalTasks(SPRINT)

In [17]:
def InplannenAdd(taskdf, dfKalenderCrew, dfKalenderRoom, dfSprint, dfAdd, prep = True, inplannen = False): 

    dfTempCrew = pickle.loads(pickle.dumps(dfKalenderCrew))
    dfTempRoom = pickle.loads(pickle.dumps(dfKalenderRoom))
    dfTempAdd = pickle.loads(pickle.dumps(dfAdd))
    gelukt = False

    if prep:
        dagIndex = 0
        weken = [1]
        maxDagIndex = 5
    else:
        dagIndex = 40
        weken = [9,10]
        maxDagIndex = 50
    # crew zelf maken
    crew = []

    if (taskdf.AdditionalReq == 0):
        
        task = taskdf.Task[5:]
        crewlijst = dfSprint[dfSprint.Task == task].iloc[0,:].Crew
        minCrew = taskdf['People.1']

        crewlijst = Hulpfuncties.somOverigeUren(crewlijst, weken, dfKalenderCrew)['crew'].iloc[:minCrew].to_list()
        crewlijst.extend([20]*(6-minCrew))
       
    elif ((len(taskdf.AdditionalReq) > 5) & (len(taskdf.AdditionalReq) < 8)):
        try:
            crewlijst = dfSprint[dfSprint.Task == taskdf.AdditionalReq].iloc[0,:].Crew
        except:
            filehandler = open('dfSprint1_eind.obj', 'rb') # Moet worden verandert met de sprint
            dfSprintVorig = pickle.load(filehandler)
            crewlijst = dfSprintVorig[dfSprintVorig.Task == taskdf.AdditionalReq].iloc[0,:].Crew
            
    elif (len(taskdf.AdditionalReq) > 8 ):
        taskA = taskdf.AdditionalReq[0:7]
        taskB = taskdf.AdditionalReq[-7:]
        taskACrew = dfSprint[dfSprint.Task == taskA].iloc[0,:].Crew
        taskBCrew = dfSprint[dfSprint.Task == taskB].iloc[0,:].Crew
        crewlijst = [20]*6
        b = list(set(taskACrew)&set(taskBCrew))
        try:
            b.remove(20)
        except:
            None
        crewlijst[0] = b[0]
        
    a = list(set(crewlijst))
    try:
        a.remove(20)
    except:
        None
    aantalCrew = len(a)
    kolom = 1
    if (int(taskdf['People.1'])==int(aantalCrew)):
        kolom = 1
    elif (int(taskdf['People.2'])==int(aantalCrew)):
        kolom = 2
    elif (int(taskdf['People.3'])==int(aantalCrew)):
        kolom = 3
    
 # tot en met hier klopt de functie
    lijstVrijeUren = [0]*16
    controle = True
    dagIndexKopie = dagIndex
    halfuur = 0
    
    if (int(round(taskdf[str("DurationExp." + str(kolom))]*2)) == 0):
        dfAdd.loc[dfAdd['Task'] == taskdf.Task, 'Voltooid'] = True
        gelukt = True
    
    else:
        while halfuur < int(round(taskdf[str("DurationExp." + str(kolom))]*2)):
            for i in crewlijst:
                if (i == 20):
                    break
                if ((float(dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1]))< float(0.5)):
                    controle = False
                    break 
                lijstVrijeUren = Hulpfuncties.CombinatieLijst(dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[1:,i+1].tolist(), lijstVrijeUren) # voor iedere op hetzelfde moment

            if ((Hulpfuncties.contains([0], lijstVrijeUren) != False) & controle):
                rijbegin , rijeind = Hulpfuncties.contains([0], lijstVrijeUren)

                kamerGevonden = False

                kamers = Hulpfuncties.ZoekKamers(taskdf.iloc[14:20], dfAdd)

                ## hier nog even naar kijken of het klopt dat er niet naar rijeind en begin wordt gekeken !TODO
                for kamer in kamers:
                    if (Hulpfuncties.contains([0], dfKalenderRoom.iloc[dagIndexKopie,:].DagRooster.dfRooster[kamer].iloc[1:].tolist()) != False):

                        if inplannen:
                            dfKalenderRoom.iloc[dagIndexKopie,:].DagRooster.dfRooster[kamer].iloc[rijbegin:rijeind+1] = taskdf.Task
                        kamerGevonden = True
                        controle = True
                        break
                    elif ((kamer == kamers[-1]) & (kamerGevonden == False)):                       
                        controle = False

                if (controle == True):
                    for i in crewlijst:
                        if i == 20:
                            break
                                                
                        dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[rijbegin+1:rijeind+2,i+1] = taskdf.Task 
                        dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1] = dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1] - float(0.5)

    #                     print(taskdf)
                        dfAdd.at[dfAdd.index[dfAdd['Task'] == taskdf.Task].tolist()[0], 'Crew'] = crewlijst
                        halfuur +=1

                        if (halfuur == (int(round(taskdf[str("DurationExp." + str(kolom))]*2)))):
                            
                            dfAdd.loc[dfAdd['Task'] == taskdf.Task, 'Voltooid'] = True
                            print(str(taskdf.Task) + "is daadwerkelijk ingepland")
                            
                            if inplannen:
                                return [True, dfKalenderCrew, dfKalenderRoom, dfAdd]
                            else:
                                return [True, dfTempCrew, dfTempRoom, dfTempAdd]

            else:

                dagIndexKopie+=1

                controle = True
                lijstVrijeUren = [0]*16
                if (dagIndexKopie == maxDagIndex):
                    break

    print(str(taskdf.Task) + " Niet ingepland of: " + str(gelukt))
    return [gelukt, dfTempCrew, dfTempRoom, dfTempAdd]

In [18]:
def InplannenOpdracht(taskdf, dagIndex, dfKalenderCrew, dfKalenderRoom, dfAddPrep, dfAddPost, dfSprint, crewA = []): 
# task moet een rij van het dataframe dfSprint zijn en een dag (als index) van een rij in dfKalenderCrew (verschil ten opzichten van 2020-1-6)
        
    dfTempCrew = pickle.loads(pickle.dumps(dfKalenderCrew))
    dfTempRoom = pickle.loads(pickle.dumps(dfKalenderRoom))
    dfTempSprint = pickle.loads(pickle.dumps(dfSprint))
    dfTempPrep = pickle.loads(pickle.dumps(dfAddPrep))
    dfTempPost = pickle.loads(pickle.dumps(dfAddPost))
    
    for crewlijst in taskdf.Mogelijkheden.dfMogelijkheden.iloc[:,1].tolist():
        a = list(set(crewlijst))
        try:
            a.remove(20)
        except:
            None
        if ((len(set(a) & set(crewA)) >= 1) | (len(crewA)==0)):
            lijstVrijeUren = [0]*16
            controle = True
            dagIndexKopie = dagIndex

            freq = 1
            while (freq <= int(taskdf.Frequency)) & (taskdf.Voltooid == False):
                if (taskdf.Eis != 'no') & (dagIndex == dagIndexKopie):
                    eis = dfSprint[dfSprint.Task == task.Eis].iloc[0,:]
                    lijstVrijeUren = [1]*int(eis["Moment Voltooid"]) + [0]*(16-int(eis["Moment Voltooid"]))
                for i in crewlijst:
                    if (i == 20):
                        break
                    if ((float(dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1]))< float(taskdf.Duration)):
                        controle = False
                        break 
                    lijstVrijeUren = Hulpfuncties.CombinatieLijst(dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[1:,i+1].tolist(), lijstVrijeUren) # voor iedere op hetzelfde moment
                duration = [0]*int(round(taskdf.Duration*2))

                if (Hulpfuncties.contains(duration, lijstVrijeUren) != False) & controle:
                    rijbegin , rijeind = Hulpfuncties.contains(duration, lijstVrijeUren)

                    kamerGevonden = False
                    for i in taskdf.Mogelijkheden.dfMogelijkheden.iloc[0,0]:
                        if (Hulpfuncties.contains(duration, dfKalenderRoom.iloc[dagIndexKopie,:].DagRooster.dfRooster[i].iloc[1:].tolist()) != False):
                            dfKalenderRoom.iloc[dagIndexKopie,:].DagRooster.dfRooster[i].iloc[rijbegin:rijeind+1] = taskdf.Task + " freq: " + str(freq)
                            kamerGevonden = True
                            controle = True
                            break
                        elif ((i == taskdf.Mogelijkheden.dfMogelijkheden.iloc[0,0][-1])& (kamerGevonden == False)):
                            controle = False
                    if (controle == True):
                        for i in crewlijst:
                            if i == 20:
                                break

                            dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[rijbegin+1:rijeind+2,i+1] = taskdf.Task + " freq: " + str(freq) # + " " + str(taskdf.Mogelijkheden.dfMogelijkheden.iloc[crewlijst,0].iloc[0])
                            dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1] = dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1] - float(taskdf.Duration)

                        if (freq == int(taskdf.Frequency)):
                            print(taskdf.Task)
                            dfPrep = pd.DataFrame(dfAddPrep[dfAddPrep['Task'].str.contains(taskdf.Task)]).reset_index(drop=True).iloc[0,:]
                            dfPost = pd.DataFrame(dfAddPost[dfAddPost['Task'].str.contains(taskdf.Task)]).reset_index(drop=True).iloc[0,:]
                            
                            dfSprint.at[dfSprint.index[dfSprint['Task'] == taskdf.Task].tolist()[0], 'Crew'] = crewlijst
                            geluktPrep, dfKalenderCrew, dfKalenderRoom, dfAddPrep = InplannenAdd(taskdf=dfPrep, dfKalenderCrew=dfKalenderCrew,
                                                                                      dfKalenderRoom=dfKalenderRoom, 
                                                                                      dfSprint=dfSprint, dfAdd=dfAddPrep, 
                                                                                      prep = True)

                            geluktPost, dfKalenderCrew, dfKalenderRoom, dfAddPost = InplannenAdd(taskdf=dfPost, dfKalenderCrew=dfKalenderCrew,
                                                                                      dfKalenderRoom=dfKalenderRoom, 
                                                                                      dfSprint=dfSprint, dfAdd=dfAddPost, 
                                                                                      prep = False)
                            
                            if (geluktPost & geluktPrep):
                                
                                geluktPrep, dfKalenderCrew, dfKalenderRoom, dfAddPrep = InplannenAdd(taskdf=dfPrep, dfKalenderCrew=dfKalenderCrew,
                                                                                      dfKalenderRoom=dfKalenderRoom, 
                                                                                      dfSprint=dfSprint, dfAdd=dfAddPrep, 
                                                                                        prep = True, inplannen=True)
                                geluktPost, dfKalenderCrew, dfKalenderRoom, dfAddPost = InplannenAdd(taskdf=dfPost, dfKalenderCrew=dfKalenderCrew,
                                                                                      dfKalenderRoom=dfKalenderRoom, 
                                                                                      dfSprint=dfSprint, dfAdd=dfAddPost, 
                                                                                      prep = False, inplannen=True)
                                
                                dfSprint.loc[dfSprint['Task'] == taskdf.Task, 'Moment Voltooid'] = rijeind +1
                                dfSprint.loc[dfSprint['Task'] == taskdf.Task, 'Dag Voltooid'] = dagIndexKopie
                                dfSprint.loc[dfSprint['Task'] == taskdf.Task, 'Voltooid'] = True
                                
                                print(str(taskdf.Task) + " is ingepland, met prep en post")
                                return [dfKalenderCrew, dfKalenderRoom, dfSprint, dfAddPrep, dfAddPost]

                        freq += 1
                else:
                    dagIndexKopie+=1

                    controle = True
                    lijstVrijeUren = [0]*16
                    if (dagIndexKopie == 10):
                        dfKalenderCrew = pickle.loads(pickle.dumps(dfTempCrew))
                        dfKalenderRoom = pickle.loads(pickle.dumps(dfTempRoom))
                        dfSprint = pickle.loads(pickle.dumps(dfTempSprint))
                        dfAddPrep = pickle.loads(pickle.dumps(dfTempPrep))
                        dfAddPost = pickle.loads(pickle.dumps(dfTempPost))
                        break
    return [dfKalenderCrew, dfKalenderRoom, dfSprint, dfAddPrep, dfAddPost]
    
    

In [19]:
dfk = pickle.loads(pickle.dumps(dfKalenderCrew))
dfr = pickle.loads(pickle.dumps(dfKalenderRoom))
dfs = pickle.loads(pickle.dumps(dfSprint))
dfprep = pickle.loads(pickle.dumps(dfSprintAddprep))
dfpost = pickle.loads(pickle.dumps(dfSprintAddpost))

In [20]:
import warnings
warnings.filterwarnings("ignore")
# misschien bij een hogere iteratie een counter bijhouden voor de tasks die niet zijn ingepland.
iteratie = 0
while (dfSprint.Voltooid.sum()< len(dfSprint)) & (iteratie < 3):
    dfSprint = dfSprint.sort_values(['Voltooid', 'Eis', 'AantalMogelijkheden'], ascending= [True, True, False]).reset_index(drop=True)
    Eisen = dfSprint.Eis.unique().tolist()
    Eisen.remove('no')

    dfSprint = (dfSprint.loc[dfSprint['Task'].isin(Eisen)].append(dfSprint.loc[~dfSprint['Task'].isin(Eisen)])).reset_index(drop=True)
    rows = dfSprint.Task.tolist()
    
    dfKalenderCrew = pickle.loads(pickle.dumps(dfk))
    dfKalenderRoom = pickle.loads(pickle.dumps(dfr))
    dfSprintAddprep = pickle.loads(pickle.dumps(dfprep))
    dfSprintAddpost = pickle.loads(pickle.dumps(dfpost))
    
    dfSprint = pickle.loads(pickle.dumps(dfs))
    dfSprint = dfSprint.set_index('Task')
    dfSprint = dfSprint.reindex(index= rows)
    dfSprint = dfSprint.reset_index()
    
    for i in range(0,len(dfSprint)):
        task = dfSprint.iloc[i,:]
        dagIndex = 5
        dfTempCrew = pickle.loads(pickle.dumps(dfKalenderCrew))
        dfTempRoom = pickle.loads(pickle.dumps(dfKalenderRoom))
        dfTempSprint = pickle.loads(pickle.dumps(dfSprint)) 
        crewA = []
        if task.Eis != 'no':
            eis = dfSprint[dfSprint.Task == task.Eis].iloc[0,:]
            if eis.Voltooid == False:    
                dfKalenderCrew, dfKalenderRoom, dfSprint, dfSprintAddprep, dfSprintAddpost = InplannenOpdracht(taskdf=eis,
                                                                   dagIndex=dagIndex, dfSprint=dfSprint,
                                                                   dfKalenderCrew=dfKalenderCrew, 
                                                                   dfKalenderRoom=dfKalenderRoom,
                                                                   dfAddPrep=dfSprintAddprep, dfAddPost=dfSprintAddpost)
            eis = dfSprint[dfSprint.Task == task.Eis].iloc[0,:]
            dagIndex = int(eis['Dag Voltooid'])
            crewA = eis.Crew
        dfKalenderCrew, dfKalenderRoom, dfSprint, dfSprintAddprep, dfSprintAddpost = InplannenOpdracht(taskdf=task,
                                                               dagIndex=dagIndex, crewA=crewA,
                                                               dfKalenderCrew=dfKalenderCrew, 
                                                               dfKalenderRoom=dfKalenderRoom,
                                                               dfAddPrep=dfSprintAddprep, dfAddPost=dfSprintAddpost,
                                                               dfSprint=dfSprint)
    iteratie += 1
    print(dfSprint.iloc[:,[0,20,21,22,23,27,28,29,30,31]])


S1C1T1A
prep S1C1T1Ais daadwerkelijk ingepland
post S1C1T1Ais daadwerkelijk ingepland
prep S1C1T1Ais daadwerkelijk ingepland
post S1C1T1Ais daadwerkelijk ingepland
S1C1T1A is ingepland, met prep en post
S1C2T2A
prep S1C2T2Ais daadwerkelijk ingepland
post S1C2T2Ais daadwerkelijk ingepland
prep S1C2T2Ais daadwerkelijk ingepland
post S1C2T2Ais daadwerkelijk ingepland
S1C2T2A is ingepland, met prep en post
S1C2T3A
prep S1C2T3Ais daadwerkelijk ingepland
post S1C2T3Ais daadwerkelijk ingepland
prep S1C2T3Ais daadwerkelijk ingepland
post S1C2T3Ais daadwerkelijk ingepland
S1C2T3A is ingepland, met prep en post
S1C3T5A
prep S1C3T5Ais daadwerkelijk ingepland
post S1C3T5A Niet ingepland of: True
prep S1C3T5Ais daadwerkelijk ingepland
post S1C3T5A Niet ingepland of: True
S1C3T5A is ingepland, met prep en post
S1C3T2A
prep S1C3T2Ais daadwerkelijk ingepland
post S1C3T2A Niet ingepland of: True
prep S1C3T2Ais daadwerkelijk ingepland
post S1C3T2A Niet ingepland of: True
S1C3T2A is ingepland, met prep e

S1C1T1B
prep S1C1T1Bis daadwerkelijk ingepland
post S1C1T1Bis daadwerkelijk ingepland
prep S1C1T1Bis daadwerkelijk ingepland
post S1C1T1Bis daadwerkelijk ingepland
S1C1T1B is ingepland, met prep en post
S1C2T2B
prep S1C2T2Bis daadwerkelijk ingepland
post S1C2T2Bis daadwerkelijk ingepland
prep S1C2T2Bis daadwerkelijk ingepland
post S1C2T2Bis daadwerkelijk ingepland
S1C2T2B is ingepland, met prep en post
S1C2T3B
prep S1C2T3B Niet ingepland of: True
post S1C2T3Bis daadwerkelijk ingepland
prep S1C2T3B Niet ingepland of: True
post S1C2T3Bis daadwerkelijk ingepland
S1C2T3B is ingepland, met prep en post
S1C3T2B
prep S1C3T2Bis daadwerkelijk ingepland
post S1C3T2Bis daadwerkelijk ingepland
prep S1C3T2Bis daadwerkelijk ingepland
post S1C3T2Bis daadwerkelijk ingepland
S1C3T2B is ingepland, met prep en post
S1C3T5B
prep S1C3T5Bis daadwerkelijk ingepland
post S1C3T5Bis daadwerkelijk ingepland
prep S1C3T5Bis daadwerkelijk ingepland
post S1C3T5Bis daadwerkelijk ingepland
S1C3T5B is ingepland, met pr

post S1C1T3is daadwerkelijk ingepland
prep S1C1T3is daadwerkelijk ingepland
post S1C1T3is daadwerkelijk ingepland
S1C1T3 is ingepland, met prep en post
S1C2T1
prep S1C2T1is daadwerkelijk ingepland
post S1C2T1 Niet ingepland of: False
S1C2T1
prep S1C2T1is daadwerkelijk ingepland
post S1C2T1 Niet ingepland of: False
S1C2T1
prep S1C2T1is daadwerkelijk ingepland
post S1C2T1 Niet ingepland of: False
S1C2T1
prep S1C2T1is daadwerkelijk ingepland
post S1C2T1 Niet ingepland of: False
S1C2T1
prep S1C2T1is daadwerkelijk ingepland
post S1C2T1is daadwerkelijk ingepland
prep S1C2T1is daadwerkelijk ingepland
post S1C2T1is daadwerkelijk ingepland
S1C2T1 is ingepland, met prep en post
S1C1T5
prep S1C1T5is daadwerkelijk ingepland
post S1C1T5is daadwerkelijk ingepland
prep S1C1T5is daadwerkelijk ingepland
post S1C1T5is daadwerkelijk ingepland
S1C1T5 is ingepland, met prep en post
S1C2T4
prep S1C2T4is daadwerkelijk ingepland
post S1C2T4 Niet ingepland of: False
S1C2T4
prep S1C2T4is daadwerkelijk ingepland

In [21]:
dfSprint.iloc[:,[0,20,21,22,23,25,27,28,29,30,31]]

,Task,MinReqCrew,MaxReqCrew,Frequency,Duration,Crew,AantalMogelijkheden,Voltooid,Eis,Moment Voltooid,Dag Voltooid
0,S1C1T1A,2.0,6.0,4.0,2.0,"[9, 0, 20, 20, 20, 20]",8.0,True,no,10,6
1,S1C2T2A,1.0,2.0,4.0,2.0,"[9, 20, 20, 20, 20, 20]",4.0,True,no,12,7
2,S1C2T3A,1.0,2.0,2.0,1.5,"[4, 20, 20, 20, 20, 20]",2.0,True,no,12,6
3,S1C3T5A,1.0,2.0,2.0,1.5,"[5, 20, 20, 20, 20, 20]",2.0,True,no,6,5
4,S1C3T2A,1.0,6.0,1.0,2.0,"[11, 20, 20, 20, 20, 20]",1.0,True,no,4,5
5,S1C3T3,2.0,6.0,4.0,2.0,"[10, 6, 20, 20, 20, 20]",8.0,True,no,10,6
6,S1C1T1B,3.0,6.0,4.0,2.0,"[11, 10, 0, 20, 20, 20]",12.0,True,S1C1T1A,12,8
7,S1C2T2B,3.0,6.0,2.0,2.0,"[9, 3, 1, 20, 20, 20]",6.0,True,S1C2T2A,8,8
8,S1C2T3B,2.0,6.0,4.0,2.0,"[8, 4, 20, 20, 20, 20]",8.0,True,S1C2T3A,4,8
9,S1C3T2B,2.0,6.0,1.0,2.0,"[11, 1, 20, 20, 20, 20]",2.0,True,S1C3T2A,10,6


In [22]:
dfSprintAddpost.iloc[:,[0,20,21,22,23,27,28,31,32,33,34]]

,Task,AdditionalReq,DurationExp.1,DurationStd.1,DurationExp.2,People.1,People.2,aantal keer,Sprint,Voltooid,Crew
0,post S1C1T1A,0,36.0,6.0,22.0,1,2,1.0,1.0,True,"[9, 20, 20, 20, 20, 20]"
1,post S1C1T1B,0,45.0,5.0,45.0,2,2,1.0,1.0,True,"[11, 0, 20, 20, 20, 20]"
2,post S1C1T2,0,60.0,10.0,60.0,2,2,1.0,1.0,False,20
3,post S1C1T3,0,36.0,6.0,22.0,1,2,1.0,1.0,True,"[2, 20, 20, 20, 20, 20]"
4,post S1C1T4,0,36.0,6.0,22.0,1,2,1.0,1.0,True,"[12, 20, 20, 20, 20, 20]"
5,post S1C1T5,0,2.0,0.0,2.0,1,1,1.0,1.0,True,"[0, 20, 20, 20, 20, 20]"
6,post S1C2T1,S1C2T1,13.0,2.0,13.0,1,0,1.0,1.0,True,"[12, 11, 7, 3, 20, 20]"
7,post S1C2T2A,0,12.0,2.0,7.0,1,2,1.0,1.0,True,"[9, 20, 20, 20, 20, 20]"
8,post S1C2T2B,0,18.0,2.0,13.5,1,2,1.0,1.0,True,"[3, 20, 20, 20, 20, 20]"
9,post S1C2T3A,0,12.0,2.0,7.0,1,2,1.0,1.0,True,"[4, 20, 20, 20, 20, 20]"


In [23]:
dfKalenderCrew.iloc[45,:].DagRooster.dfRooster

,Tijd,Alyssa,Chris,Dennis,Iris,Jurgen,Lois,Melanie,Mitch,Rudy,Shane,Wendy,William,Winston
0,urenTeGaan,3.5,0,0,4,0,0,4,4,7,0,7,1.5,0
1,09:00:00,post S1C2T5,0,post S1C1T3,0,0,post S1C2T4,0,0,0,post S1C1T1A,0,post S1C3T2B,post S1C1T4
2,09:30:00,0,0,post S1C1T3,0,0,post S1C2T4,0,0,0,post S1C1T1A,0,post S1C3T2B,post S1C1T4
3,10:00:00,0,0,post S1C1T3,0,0,post S1C2T4,0,0,0,post S1C1T1A,0,post S1C3T2B,post S1C1T4
4,10:30:00,0,0,post S1C1T3,0,0,post S1C2T4,0,0,0,post S1C1T1A,0,post S1C3T2B,post S1C1T4
5,11:00:00,0,0,post S1C1T3,0,0,post S1C2T4,0,0,0,post S1C1T1A,0,post S1C3T2B,post S1C1T4
6,11:30:00,0,0,post S1C1T3,0,0,post S1C2T4,0,0,0,post S1C1T1A,0,0,post S1C1T4
7,12:30:00,0,0,post S1C1T3,0,0,post S1C2T4,0,0,0,post S1C1T1A,0,0,post S1C1T4
8,13:00:00,0,0,post S1C1T3,0,0,post S1C2T4,0,0,0,post S1C1T1A,0,0,post S1C1T4
9,13:30:00,post S1C3T1,0,post S1C3T1,post S1C2T1,0,post S1C2T4,post S1C3T1,post S1C2T1,0,post S1C1T1A,0,post S1C2T1,post S1C2T1


In [24]:
dfKalenderCrew.iloc[46,:].DagRooster.dfRooster

,Tijd,Alyssa,Chris,Dennis,Iris,Jurgen,Lois,Melanie,Mitch,Rudy,Shane,Wendy,William,Winston
0,urenTeGaan,0,4,0,4,4,0,0,4,4,0,4,3.5,3.5
1,09:00:00,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up
2,09:30:00,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up
3,10:00:00,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up
4,10:30:00,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up
5,11:00:00,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up
6,11:30:00,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up
7,12:30:00,post S1C3T1,0,post S1C3T1,0,0,post S1C2T4,post S1C3T1,0,0,post S1C1T1A,0,post S1C2T1,post S1C2T1
8,13:00:00,post S1C3T1,0,post S1C3T1,0,0,post S1C2T4,post S1C3T1,0,0,post S1C1T1A,0,0,0
9,13:30:00,post S1C3T1,0,post S1C3T1,0,0,post S1C2T4,post S1C3T1,0,0,post S1C1T1A,0,0,0


In [25]:
dfKalenderCrew.iloc[47,:].DagRooster.dfRooster

,Tijd,Alyssa,Chris,Dennis,Iris,Jurgen,Lois,Melanie,Mitch,Rudy,Shane,Wendy,William,Winston
0,urenTeGaan,0,0,7,0,7,1,0,0,7,0,0,7,7
1,09:00:00,0,0,0,0,0,post S1C2T4,0,0,0,post S1C2T2A,0,0,0
2,09:30:00,0,0,0,0,0,post S1C2T4,0,0,0,post S1C2T2A,0,0,0
3,10:00:00,0,0,0,0,0,post S1C2T4,0,0,0,post S1C2T2A,0,0,0
4,10:30:00,0,0,0,0,0,post S1C2T4,0,0,0,post S1C2T2A,0,0,0
5,11:00:00,0,0,0,0,0,post S1C2T4,0,0,0,post S1C2T2A,0,0,0
6,11:30:00,0,0,0,0,0,post S1C2T4,0,0,0,post S1C2T2A,0,0,0
7,12:30:00,0,0,0,0,0,post S1C2T4,0,0,0,post S1C2T2A,0,0,0
8,13:00:00,0,0,0,0,0,post S1C2T4,0,0,0,post S1C2T2A,0,0,0
9,13:30:00,0,0,0,0,0,post S1C2T4,0,0,0,post S1C2T2A,0,0,0


In [26]:
dfKalenderCrew.iloc[48,:].DagRooster.dfRooster

,Tijd,Alyssa,Chris,Dennis,Iris,Jurgen,Lois,Melanie,Mitch,Rudy,Shane,Wendy,William,Winston
0,urenTeGaan,3,7,2.5,7,7,7,2.5,7,7,2,7,7,7
1,09:00:00,post S1C3T1,0,post S1C3T1,0,0,0,post S1C3T1,0,0,post S1C2T2A,0,0,0
2,09:30:00,post S1C3T1,0,post S1C3T1,0,0,0,post S1C3T1,0,0,post S1C2T2A,0,0,0
3,10:00:00,post S1C3T1,0,post S1C3T1,0,0,0,post S1C3T1,0,0,post S1C2T2A,0,0,0
4,10:30:00,post S1C3T1,0,post S1C3T1,0,0,0,post S1C3T1,0,0,post S1C2T2A,0,0,0
5,11:00:00,post S1C3T1,0,post S1C3T1,0,0,0,post S1C3T1,0,0,post S1C2T2A,0,0,0
6,11:30:00,post S1C3T1,0,post S1C3T1,0,0,0,post S1C3T1,0,0,post S1C2T2A,0,0,0
7,12:30:00,post S1C3T1,0,post S1C3T1,0,0,0,post S1C3T1,0,0,post S1C2T2A,0,0,0
8,13:00:00,post S1C3T1,0,post S1C3T1,0,0,0,post S1C3T1,0,0,post S1C2T2A,0,0,0
9,13:30:00,0,0,post S1C3T1,0,0,0,post S1C3T1,0,0,post S1C2T2A,0,0,0


In [27]:
dfKalenderCrew.iloc[49,:].DagRooster.dfRooster

,Tijd,Alyssa,Chris,Dennis,Iris,Jurgen,Lois,Melanie,Mitch,Rudy,Shane,Wendy,William,Winston
0,urenTeGaan,7,0,7,0,0,7,7,7,7,0,0,7,7
1,09:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0
2,09:30:00,0,0,0,0,0,0,0,0,0,0,0,0,0
3,10:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0
4,10:30:00,0,0,0,0,0,0,0,0,0,0,0,0,0
5,11:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0
6,11:30:00,0,0,0,0,0,0,0,0,0,0,0,0,0
7,12:30:00,0,0,0,0,0,0,0,0,0,0,0,0,0
8,13:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0
9,13:30:00,0,0,0,0,0,0,0,0,0,0,0,0,0


In [28]:
dfKalenderCrew.iloc[0,:].DagRooster.dfRooster

,Tijd,Alyssa,Chris,Dennis,Iris,Jurgen,Lois,Melanie,Mitch,Rudy,Shane,Wendy,William,Winston
0,urenTeGaan,0,0,0,7,0,0,5,7,0,0,7,0,0
1,09:00:00,prep S1C1T1B,0,prep S1C3T1,0,0,prep S1C3T5A,prep S1C3T3,0,prep S1C1T3,prep S1C1T1A,0,prep S1C3T2A,prep S1C1T4
2,09:30:00,prep S1C1T1B,0,prep S1C3T1,0,0,prep S1C3T5A,prep S1C3T3,0,prep S1C1T3,prep S1C1T1A,0,prep S1C3T2A,prep S1C1T4
3,10:00:00,prep S1C1T1B,0,prep S1C3T1,0,0,prep S1C3T5A,prep S1C3T3,0,prep S1C1T3,prep S1C1T1A,0,prep S1C3T2A,prep S1C1T4
4,10:30:00,prep S1C1T1B,0,prep S1C3T1,0,0,prep S1C3T5A,prep S1C3T3,0,prep S1C1T3,prep S1C1T1A,0,prep S1C3T2A,prep S1C1T4
5,11:00:00,prep S1C1T1B,0,prep S1C3T1,0,0,prep S1C3T5A,0,0,prep S1C1T3,prep S1C1T1A,0,prep S1C3T2A,prep S1C1T4
6,11:30:00,prep S1C1T1B,0,prep S1C3T1,0,0,prep S1C3T5A,0,0,prep S1C1T3,prep S1C1T1A,0,prep S1C3T2A,prep S1C1T4
7,12:30:00,prep S1C1T1B,0,prep S1C3T1,0,0,prep S1C3T5A,0,0,prep S1C1T3,prep S1C1T1A,0,prep S1C3T2A,prep S1C1T4
8,13:00:00,prep S1C1T1B,0,prep S1C3T1,0,0,prep S1C3T5A,0,0,prep S1C1T3,prep S1C1T1A,0,prep S1C3T2A,prep S1C1T4
9,13:30:00,prep S1C1T1B,0,prep S1C3T1,0,0,prep S1C3T5A,0,0,prep S1C1T3,prep S1C1T1A,0,prep S1C3T2A,prep S1C1T4


In [29]:
dfKalenderCrew.iloc[2,:].DagRooster.dfRooster

,Tijd,Alyssa,Chris,Dennis,Iris,Jurgen,Lois,Melanie,Mitch,Rudy,Shane,Wendy,William,Winston
0,urenTeGaan,0,0,0,0,4,4,0,0,7,0,0,4,0
1,09:00:00,0,0,prep S1C3T1,0,prep S1C2T3A,prep S1C3T5B,0,0,0,prep S1C2T2A,0,prep S1C3T2B,prep S1C2T4
2,09:30:00,0,0,prep S1C3T1,0,prep S1C2T3A,prep S1C3T5B,0,0,0,prep S1C2T2A,0,prep S1C3T2B,prep S1C2T4
3,10:00:00,0,0,prep S1C3T1,0,prep S1C2T3A,prep S1C3T5B,0,0,0,prep S1C2T2A,0,prep S1C3T2B,prep S1C2T4
4,10:30:00,0,0,prep S1C3T1,0,prep S1C2T3A,prep S1C3T5B,0,0,0,prep S1C2T2A,0,prep S1C3T2B,prep S1C2T4
5,11:00:00,0,0,prep S1C3T1,0,prep S1C2T3A,prep S1C3T5B,0,0,0,prep S1C2T2A,0,prep S1C3T2B,prep S1C2T4
6,11:30:00,0,0,prep S1C3T1,0,prep S1C2T3A,prep S1C3T5B,0,0,0,prep S1C2T2A,0,prep S1C3T2B,prep S1C2T4
7,12:30:00,0,0,prep S1C3T1,0,0,0,0,0,0,prep S1C2T2B,0,0,prep S1C2T4
8,13:00:00,0,0,prep S1C3T1,0,0,0,0,0,0,prep S1C2T2B,0,0,prep S1C2T4
9,13:30:00,0,0,prep S1C3T1,0,0,0,0,0,0,prep S1C2T2B,0,0,prep S1C2T4


In [30]:
dfKalenderCrew.iloc[3,:].DagRooster.dfRooster

,Tijd,Alyssa,Chris,Dennis,Iris,Jurgen,Lois,Melanie,Mitch,Rudy,Shane,Wendy,William,Winston
0,urenTeGaan,7,7,7,7,7,7,7,4,7,2,7,7,4
1,09:00:00,0,0,0,0,0,0,0,prep S1C2T1,0,prep S1C2T2B,0,0,prep S1C2T1
2,09:30:00,0,0,0,0,0,0,0,prep S1C2T1,0,prep S1C2T2B,0,0,prep S1C2T1
3,10:00:00,0,0,0,0,0,0,0,prep S1C2T1,0,prep S1C2T2B,0,0,prep S1C2T1
4,10:30:00,0,0,0,0,0,0,0,prep S1C2T1,0,prep S1C2T2B,0,0,prep S1C2T1
5,11:00:00,0,0,0,0,0,0,0,prep S1C2T1,0,prep S1C2T2B,0,0,prep S1C2T1
6,11:30:00,0,0,0,0,0,0,0,prep S1C2T1,0,prep S1C2T2B,0,0,prep S1C2T1
7,12:30:00,0,0,0,0,0,0,0,0,0,prep S1C2T2B,0,0,0
8,13:00:00,0,0,0,0,0,0,0,0,0,prep S1C2T2B,0,0,0
9,13:30:00,0,0,0,0,0,0,0,0,0,prep S1C2T2B,0,0,0


In [31]:
dfKalenderCrew.iloc[4,:].DagRooster.dfRooster

,Tijd,Alyssa,Chris,Dennis,Iris,Jurgen,Lois,Melanie,Mitch,Rudy,Shane,Wendy,William,Winston
0,urenTeGaan,7,0,7,0,0,7,7,7,7,0,0,7,7
1,09:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0
2,09:30:00,0,0,0,0,0,0,0,0,0,0,0,0,0
3,10:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0
4,10:30:00,0,0,0,0,0,0,0,0,0,0,0,0,0
5,11:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0
6,11:30:00,0,0,0,0,0,0,0,0,0,0,0,0,0
7,12:30:00,0,0,0,0,0,0,0,0,0,0,0,0,0
8,13:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0
9,13:30:00,0,0,0,0,0,0,0,0,0,0,0,0,0


In [32]:
Controleren.EvaluateProgram(dfSprint,dfk,dfKalenderCrew)

Het aantal uur dat nog over is: 203.5
Van het totaal aantal uren: 325
Het aantal uur dat moest worden ingepland: 250.0
Het aantal opdrachten dat niet zijn ingepland: 1


In [33]:
dfKalenderRoom.iloc[5,:].DagRooster.dfRooster

,Tijd,assembly A,assembly B,workshop A,workshop B,workshop C,workspace A,workspace B,workspace C,technical A,...,office A4,office A5,office B1,office B2,office B3,office B4,office C1,office C2,office C3,office C4
0,09:00:00,0,0,S1C3T2A freq: 1,0,0,S1C1T3 freq: 1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,09:30:00,0,0,S1C3T2A freq: 1,0,0,S1C1T3 freq: 1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10:00:00,0,0,S1C3T2A freq: 1,0,0,S1C1T3 freq: 1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10:30:00,0,0,S1C3T2A freq: 1,0,0,S1C1T3 freq: 1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,11:00:00,S1C2T1 freq: 1,0,S1C3T5A freq: 2,0,0,S1C1T3 freq: 2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,11:30:00,S1C2T1 freq: 1,0,S1C3T5A freq: 2,0,0,S1C1T3 freq: 2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,12:30:00,S1C2T1 freq: 1,0,0,0,0,S1C1T3 freq: 2,0,0,S1C3T5B freq: 1,...,0,0,0,0,0,0,0,0,0,0
7,13:00:00,S1C2T1 freq: 1,0,0,0,0,S1C1T3 freq: 2,0,0,S1C3T5B freq: 1,...,0,0,0,0,0,0,0,0,0,0
8,13:30:00,0,0,0,0,0,S1C1T3 freq: 3,0,0,S1C3T5B freq: 1,...,0,0,0,0,0,0,0,0,0,0
9,14:00:00,0,0,0,0,0,S1C1T3 freq: 3,0,0,S1C3T5B freq: 1,...,0,0,0,0,0,0,0,0,0,0


!TODO 99% betrouwbaarheidsinterval

Het bedrijf werkt in sprints. Om precies te zijn: 4 sprints van elk 10 weken. De weken
2 t/m 8 van elke sprint zijn de zogeheten projectweken

In [34]:
# # het kopieren van de roosters. Deze zijn voor dit gedeelte gelijk in week 2 tot en met 8. 


for i in range(1,8):
    for j in range(5,10):
        dfKalenderRoom.iloc[i*5 + j,:].DagRooster.dfRooster = pickle.loads(pickle.dumps(dfKalenderRoom.iloc[j,:].DagRooster.dfRooster))
        dfKalenderCrew.iloc[i*5 + j,:].DagRooster.dfRooster = pickle.loads(pickle.dumps(dfKalenderCrew.iloc[j,:].DagRooster.dfRooster))
    

!TODO de mensen die de prep uitvoeren, moeten ook de projecttasks en post doen.

Wanneer kan er met de prep check-up worden begonnen en wanneer kan de check-up worden gedaan?

In [35]:
# back-up maken van de verschillende dataframes zodat deze kunnen worden terug gezet.

dfk = pickle.loads(pickle.dumps(dfKalenderCrew))
dfs = pickle.loads(pickle.dumps(dfSprint))
dfr = pickle.loads(pickle.dumps(dfKalenderRoom))


!TODO kiezen van de crew director

In [36]:
# for i in range(0,len(dfSprintAddprep)):
#     task = dfSprintAddprep.iloc[i,:]

#     dfTempCrew = pickle.loads(pickle.dumps(dfKalenderCrew))
#     dfTempRoom = pickle.loads(pickle.dumps(dfKalenderRoom))
#     print(task.Task)
#     dfKalenderCrew, dfKalenderRoom = InplannenAdd(task, dfKalenderCrew, dfKalenderRoom, 
#                                                       dfSprint=dfSprint, dfAdd= dfSprintAddprep)

In [37]:
# for i in range(0,len(dfSprintAddpost)):
#     task = dfSprintAddpost.iloc[i,:]

#     dfTempCrew = pickle.loads(pickle.dumps(dfKalenderCrew))
#     dfTempRoom = pickle.loads(pickle.dumps(dfKalenderRoom))
#     print(task.Task)
#     dfKalenderCrew, dfKalenderRoom = InplannenAdd(task, dfKalenderCrew, dfKalenderRoom, 
#                                                       dfSprint=dfSprint, dfAdd= dfSprintAddpost)

In [38]:
dfKalenderCrew.iloc[2,:].DagRooster.dfRooster

,Tijd,Alyssa,Chris,Dennis,Iris,Jurgen,Lois,Melanie,Mitch,Rudy,Shane,Wendy,William,Winston
0,urenTeGaan,0,0,0,0,4,4,0,0,7,0,0,4,0
1,09:00:00,0,0,prep S1C3T1,0,prep S1C2T3A,prep S1C3T5B,0,0,0,prep S1C2T2A,0,prep S1C3T2B,prep S1C2T4
2,09:30:00,0,0,prep S1C3T1,0,prep S1C2T3A,prep S1C3T5B,0,0,0,prep S1C2T2A,0,prep S1C3T2B,prep S1C2T4
3,10:00:00,0,0,prep S1C3T1,0,prep S1C2T3A,prep S1C3T5B,0,0,0,prep S1C2T2A,0,prep S1C3T2B,prep S1C2T4
4,10:30:00,0,0,prep S1C3T1,0,prep S1C2T3A,prep S1C3T5B,0,0,0,prep S1C2T2A,0,prep S1C3T2B,prep S1C2T4
5,11:00:00,0,0,prep S1C3T1,0,prep S1C2T3A,prep S1C3T5B,0,0,0,prep S1C2T2A,0,prep S1C3T2B,prep S1C2T4
6,11:30:00,0,0,prep S1C3T1,0,prep S1C2T3A,prep S1C3T5B,0,0,0,prep S1C2T2A,0,prep S1C3T2B,prep S1C2T4
7,12:30:00,0,0,prep S1C3T1,0,0,0,0,0,0,prep S1C2T2B,0,0,prep S1C2T4
8,13:00:00,0,0,prep S1C3T1,0,0,0,0,0,0,prep S1C2T2B,0,0,prep S1C2T4
9,13:30:00,0,0,prep S1C3T1,0,0,0,0,0,0,prep S1C2T2B,0,0,prep S1C2T4


In [39]:
dfKalenderCrew.iloc[40,:].DagRooster.dfRooster

,Tijd,Alyssa,Chris,Dennis,Iris,Jurgen,Lois,Melanie,Mitch,Rudy,Shane,Wendy,William,Winston
0,urenTeGaan,0,0,1,5,0,0,1,5,1,1,1,3,5
1,09:00:00,S1C1T1A freq: 1,0,S1C1T3 freq: 1,0,0,S1C3T5A freq: 1,S1C3T3 freq: 1,0,S1C1T3 freq: 1,S1C1T1A freq: 1,S1C3T3 freq: 1,S1C3T2A freq: 1,0
2,09:30:00,S1C1T1A freq: 1,0,S1C1T3 freq: 1,0,0,S1C3T5A freq: 1,S1C3T3 freq: 1,0,S1C1T3 freq: 1,S1C1T1A freq: 1,S1C3T3 freq: 1,S1C3T2A freq: 1,0
3,10:00:00,S1C1T1A freq: 1,0,S1C1T3 freq: 1,0,0,S1C3T5A freq: 1,S1C3T3 freq: 1,0,S1C1T3 freq: 1,S1C1T1A freq: 1,S1C3T3 freq: 1,S1C3T2A freq: 1,0
4,10:30:00,S1C1T1A freq: 1,0,S1C1T3 freq: 1,0,0,S1C3T5A freq: 2,S1C3T3 freq: 1,0,S1C1T3 freq: 1,S1C1T1A freq: 1,S1C3T3 freq: 1,S1C3T2A freq: 1,0
5,11:00:00,S1C1T1A freq: 2,0,S1C1T3 freq: 2,S1C2T1 freq: 1,0,S1C3T5A freq: 2,S1C3T3 freq: 2,S1C2T1 freq: 1,S1C1T3 freq: 2,S1C1T1A freq: 2,S1C3T3 freq: 2,S1C2T1 freq: 1,S1C2T1 freq: 1
6,11:30:00,S1C1T1A freq: 2,0,S1C1T3 freq: 2,S1C2T1 freq: 1,0,S1C3T5A freq: 2,S1C3T3 freq: 2,S1C2T1 freq: 1,S1C1T3 freq: 2,S1C1T1A freq: 2,S1C3T3 freq: 2,S1C2T1 freq: 1,S1C2T1 freq: 1
7,12:30:00,S1C1T1A freq: 2,0,S1C1T3 freq: 2,S1C2T1 freq: 1,0,S1C3T5B freq: 1,S1C3T3 freq: 2,S1C2T1 freq: 1,S1C1T3 freq: 2,S1C1T1A freq: 2,S1C3T3 freq: 2,S1C2T1 freq: 1,S1C2T1 freq: 1
8,13:00:00,S1C1T1A freq: 2,0,S1C1T3 freq: 2,S1C2T1 freq: 1,0,S1C3T5B freq: 1,S1C3T3 freq: 2,S1C2T1 freq: 1,S1C1T3 freq: 2,S1C1T1A freq: 2,S1C3T3 freq: 2,S1C2T1 freq: 1,S1C2T1 freq: 1
9,13:30:00,S1C1T1A freq: 3,0,S1C1T3 freq: 3,0,0,S1C3T5B freq: 1,S1C3T3 freq: 3,0,S1C1T3 freq: 3,S1C1T1A freq: 3,S1C3T3 freq: 3,0,0


In [40]:
filehandler = open('dfSprint1_eind.obj', 'wb') 
pickle.dump(dfSprint, filehandler)

filehandler = open('dfKalenderCrew1.obj', 'wb') 
pickle.dump(dfKalenderCrew, filehandler)

filehandler = open('dfKalenderRoom1.obj', 'wb') 
pickle.dump(dfKalenderRoom, filehandler)

filehandler = open('dfSprintAddprep1.obj', 'wb') 
pickle.dump(dfSprintAddprep, filehandler)

filehandler = open('dfSprintAddpost1.obj', 'wb') 
pickle.dump(dfSprintAddpost, filehandler)
